In [3]:
import os
from os import path
import cv2
import imutils
import numpy as np

In [4]:
def dataset_creation():
    bg = None

    t,r,b,l=100,250,425,575
    num_frames=0

    cur_mode=None
    count=0
    limit=500
    method=1

    cam=cv2.VideoCapture(0)
    while(cam.isOpened()):
        _,frame=cam.read()
        if frame is not None:
            frame=imutils.resize(frame,width=700)
            frame=cv2.flip(frame,1)
            clone=frame.copy()
            roi=frame[t:b,r:l]
            cv2.rectangle(clone,(l,t),(r,b),(0,255,0),2)
            hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
            lh = 0
            ls = 0
            lv = 0
            uh = 255
            us = 32
            uv = 255

            l_b = np.array([lh, ls, lv])
            u_b = np.array([uh, us, uv])

            mask = cv2.inRange(hsv, l_b, u_b)
            #cv2.imshow('mask', mask)
            mask = cv2.bitwise_not(mask)
            mask = cv2.medianBlur(mask, 5)
            mask = cv2.addWeighted(mask, 0.5, mask, 0.5, 0.0)
            kernel = np.ones((5, 5), np.uint8)
            mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

            res = cv2.bitwise_and(roi, roi, mask=mask)
            res=cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
            #cv2.imshow('res', res)
            high_thresh, thresh_im = cv2.threshold(res, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            lowThresh = 0.5 * high_thresh
            res = cv2.Canny(res, lowThresh, high_thresh)
            cv2.imshow('canny',res)

            if cur_mode!=-1 and cur_mode!=255 and cur_mode is not None:
               
                file_path = 'C:\\Users\\ssa20\\Documents\\Major_project\\new\\Major_Code\\Saved Dataset\\'+str(chr(cur_mode))                
                if not path.exists(file_path):
                    os.makedirs(file_path)
                if(count<=limit):
                    cv2.imwrite(file_path+'\\'+str(count)+'.jpg',res)
                    print(count)
                    if(count==limit):
                        print("Completed")
                count+=1

            num_frames+=1
            cv2.putText(clone, "Put your hand gestures in the rectangle", (5, 25), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
            cv2.putText(clone, "Press 'exc' to exit.", (5, 50), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
            cv2.putText(clone, "Press the key to create dataset", (5, 75), cv2.FONT_HERSHEY_COMPLEX, 0.5, (0, 0, 0))
            cv2.imshow("Video Feed",clone)

            k=cv2.waitKey(1)& 0xFF
            if (k==27):
               break
            if(k!=-1 and k!=255 and k!=cur_mode):
                cur_mode=k
                count = 0

    cam.release()
    cv2.destroyAllWindows()

In [6]:
#dataset_creation()